In [1]:
#import dependecies
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings('ignore')

import datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

#import dependencies for ML models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score
from sklearn.svm import SVR
from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve


#import dependencies for graphs
import numpy as np
import matplotlib.pyplot as plt 
from scipy import stats
import seaborn as sns; sns.set_theme(color_codes=True)
sns.set_context("paper")


In [2]:
#connect to a sqlite3 db
conn = sqlite3.connect(r'/Users/rosaicelaroman/Desktop/portfolio_work/earthquake_modeling/Database/db/earthquake_db')
print("Connected to database...")

Connected to database...


In [3]:
model_df = pd.read_sql_query("SELECT * FROM earthquake_raw", conn)
model_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.10750,1.100000,1.30,ml,8.0,149.46,0.875,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.81800,7.300000,1.40,ml,11.0,154.64,0.119,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.88520,30.000000,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.76250,111.300000,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.76570,1.900000,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28330,2022-01-30T03:46:53.207Z,63.278200,-151.25170,7.400000,1.00,ml,NaN,NaN,NaN,0.6600,...,2022-02-25T00:07:27.831Z,"37 km SE of Denali National Park, Alaska",earthquake,NaN,0.20,NaN,NaN,reviewed,ak,ak
28331,2022-01-30T03:44:08.272Z,67.671500,-160.82990,6.200000,2.90,ml,NaN,NaN,NaN,0.5700,...,2022-02-25T03:00:49.040Z,"79 km NNW of Kiana, Alaska",earthquake,NaN,0.60,NaN,NaN,reviewed,ak,ak
28332,2022-01-30T03:37:46.570Z,19.221167,-155.38916,31.940001,1.83,md,43.0,152.00,NaN,0.1300,...,2022-01-30T03:40:57.760Z,"9 km ENE of Pāhala, Hawaii",earthquake,0.59,1.16,0.620,5.0,automatic,hv,hv
28333,2022-01-30T03:32:41.102Z,2.757300,128.23460,53.450000,4.50,mb,NaN,194.00,2.154,1.0300,...,2022-02-05T01:40:12.040Z,"Halmahera, Indonesia",earthquake,12.30,10.40,0.272,5.0,reviewed,us,us


In [4]:
#converting time to datetime
model_df['time'] = pd.to_datetime(model_df['time'])
model_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06 01:42:39.855000+00:00,37.384000,-117.10750,1.100000,1.30,ml,8.0,149.46,0.875,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290,4.0,automatic,nn,nn
1,2022-01-06 01:31:33.698000+00:00,38.181000,-117.81800,7.300000,1.40,ml,11.0,154.64,0.119,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460,5.0,automatic,nn,nn
2,2022-01-06 01:24:20.083000+00:00,61.340700,-149.88520,30.000000,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06 01:15:15.851000+00:00,62.870300,-150.76250,111.300000,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06 01:10:44.629000+00:00,64.014500,-148.76570,1.900000,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28330,2022-01-30 03:46:53.207000+00:00,63.278200,-151.25170,7.400000,1.00,ml,NaN,NaN,NaN,0.6600,...,2022-02-25T00:07:27.831Z,"37 km SE of Denali National Park, Alaska",earthquake,NaN,0.20,NaN,NaN,reviewed,ak,ak
28331,2022-01-30 03:44:08.272000+00:00,67.671500,-160.82990,6.200000,2.90,ml,NaN,NaN,NaN,0.5700,...,2022-02-25T03:00:49.040Z,"79 km NNW of Kiana, Alaska",earthquake,NaN,0.60,NaN,NaN,reviewed,ak,ak
28332,2022-01-30 03:37:46.570000+00:00,19.221167,-155.38916,31.940001,1.83,md,43.0,152.00,NaN,0.1300,...,2022-01-30T03:40:57.760Z,"9 km ENE of Pāhala, Hawaii",earthquake,0.59,1.16,0.620,5.0,automatic,hv,hv
28333,2022-01-30 03:32:41.102000+00:00,2.757300,128.23460,53.450000,4.50,mb,NaN,194.00,2.154,1.0300,...,2022-02-05T01:40:12.040Z,"Halmahera, Indonesia",earthquake,12.30,10.40,0.272,5.0,reviewed,us,us


In [5]:
#adding column of converted time to datetime
model_df['Unix Time'] = pd.to_datetime(model_df['time']).astype(int)/10**9
model_df.head(5)

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,Unix Time
0,2022-01-06 01:42:39.855000+00:00,37.3840,-117.1075,1.1,1.3,ml,8.0,149.46,0.875,0.2175,...,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.1,0.29,4.0,automatic,nn,nn,1.641433e+09
1,2022-01-06 01:31:33.698000+00:00,38.1810,-117.8180,7.3,1.4,ml,11.0,154.64,0.119,0.1017,...,"34 km SE of Mina, Nevada",earthquake,NaN,2.0,0.46,5.0,automatic,nn,nn,1.641433e+09
2,2022-01-06 01:24:20.083000+00:00,61.3407,-149.8852,30.0,1.5,ml,NaN,NaN,NaN,0.7500,...,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.2,NaN,NaN,automatic,ak,ak,1.641432e+09
3,2022-01-06 01:15:15.851000+00:00,62.8703,-150.7625,111.3,2.1,ml,NaN,NaN,NaN,0.3200,...,"41 km N of Petersville, Alaska",earthquake,NaN,0.8,NaN,NaN,automatic,ak,ak,1.641432e+09
4,2022-01-06 01:10:44.629000+00:00,64.0145,-148.7657,1.9,1.5,ml,NaN,NaN,NaN,0.7200,...,"17 km E of Ferry, Alaska",earthquake,NaN,0.6,NaN,NaN,automatic,ak,ak,1.641431e+09


In [6]:
for col in model_df.columns:
    print(col)

time
latitude
longitude
depth
mag
magType
nst
gap
dmin
rms
net
id
updated
place
type
horizontalError
depthError
magError
magNst
status
locationSource
magSource
Unix Time


In [7]:
#drop columns not needed
cleaned_df = model_df.drop(columns=['id', 'nst','rms','magNst','net', 'time', 'magType', 'gap','dmin', 'updated', 'place', 'type', 'horizontalError', 'depthError', 'magError', 'status', 'locationSource', 'magSource'])
cleaned_df

,latitude,longitude,depth,mag,Unix Time
0,37.384000,-117.10750,1.100000,1.30,1.641433e+09
1,38.181000,-117.81800,7.300000,1.40,1.641433e+09
2,61.340700,-149.88520,30.000000,1.50,1.641432e+09
3,62.870300,-150.76250,111.300000,2.10,1.641432e+09
4,64.014500,-148.76570,1.900000,1.50,1.641431e+09
...,...,...,...,...,...
28330,63.278200,-151.25170,7.400000,1.00,1.643514e+09
28331,67.671500,-160.82990,6.200000,2.90,1.643514e+09
28332,19.221167,-155.38916,31.940001,1.83,1.643514e+09
28333,2.757300,128.23460,53.450000,4.50,1.643514e+09


In [8]:
#find if any column has NaNs
cleaned_df.isnull().sum()

latitude     0
longitude    0
depth        0
mag          6
Unix Time    0
dtype: int64

In [9]:
#drop NaNs
cleaned_df_dropped = cleaned_df.dropna()

In [10]:
#verify dropped NaNs
cleaned_df_dropped.isnull().sum()

latitude     0
longitude    0
depth        0
mag          0
Unix Time    0
dtype: int64

In [11]:
#establish features to be used in model. X will use: latitude, longitude, depth and unix time. Y will be magnitude
X = cleaned_df_dropped.drop(['mag'], axis = 1)

y = cleaned_df_dropped[['mag']]

In [12]:
#checking number of columns and rows for eacg feature
X.shape, y.shape

((28329, 4), (28329, 1))

In [13]:
#scale and transform the data
sc_X = StandardScaler()
sc_y = StandardScaler()
X = sc_X.fit_transform(X)
y = sc_y.fit_transform(y)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

Extra Trees Regressor

In [15]:
#create and run model: First with 50 estimators
scores = []
estimators_lst=[]

for i in range(1,50):
    model = ExtraTreesRegressor(n_estimators=i+1, random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test,y_pred)

    r2 = r2_score(y_test, y_pred)
    score = model.score(X_test, y_test)
    estimators = i+1
    scores.append(score)
    estimators_lst.append(score)

    print("n_estimators={}:".format(estimators))
    print("\tMean Squared Error:", mse, end = ", ")
    print("\tR2 score:", r2, end = ", ")
    print("\tModel score:", score)
max_score = max(scores)
print("Max score:", max_score)
index_max_score = scores.index(max_score) #find the position in the list for max score to get the number of estimators
best_score = max_score
best_model = ExtraTreesRegressor(n_estimators=index_max_score+2, random_state=0)
best_model.fit(X_train, y_train)
best_model_score = best_model.score(X_test, y_test)
print("Best model score: ", best_model_score)


n_estimators=2:
	Mean Squared Error: 0.21028899698021208, 	R2 score: 0.7925981274505838, 	Model score: 0.7925981274505838
n_estimators=3:
	Mean Squared Error: 0.1848344120051422, 	R2 score: 0.8177032383437348, 	Model score: 0.8177032383437348
n_estimators=4:
	Mean Squared Error: 0.1735920792742039, 	R2 score: 0.8287912215178597, 	Model score: 0.8287912215178597
n_estimators=5:
	Mean Squared Error: 0.1671274993191277, 	R2 score: 0.8351670472014754, 	Model score: 0.8351670472014754
n_estimators=6:
	Mean Squared Error: 0.16258598636752924, 	R2 score: 0.8396462082793019, 	Model score: 0.8396462082793019
n_estimators=7:
	Mean Squared Error: 0.15959214357214335, 	R2 score: 0.8425989476560561, 	Model score: 0.8425989476560561
n_estimators=8:
	Mean Squared Error: 0.1569536245635218, 	R2 score: 0.8452012415991712, 	Model score: 0.8452012415991712
n_estimators=9:
	Mean Squared Error: 0.15503735132541008, 	R2 score: 0.8470912057133568, 	Model score: 0.8470912057133568
n_estimators=10:
	Mean Squar

In [16]:
#create and run model: Now with 200 max estimators if no improvement of the score is found
scores = []
counter = 0
import time
start_time = time.perf_counter()
for i in range(1,200):
    model = ExtraTreesRegressor(n_estimators=i+1, random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test,y_pred)

    r2 = r2_score(y_test, y_pred)
    score = model.score(X_test, y_test)
    estimators = i+1
    current_score = score
    num_estimators = estimators

    print("n_estimators={}:".format(estimators))
    print("\tMean Squared Error:", mse, end = ", ")
    print("\tR2:", r2, end = ", ")
    print("\tModel score:", score)
    if current_score > best_score:
        best_score = current_score
        best_model = model
        num_estimators=i+1
        print('Improvement found!')
        counter = 0
    else:
        counter += 1
    if counter > 50:
        break

end_time = time.perf_counter()
execution_time = end_time - start_time

print(f'Best score found: {best_score:.3f}.')
print(f'Number of estimators: {num_estimators}.')
print(f'Code execution time: {execution_time:.3f} seconds.')

n_estimators=2:
	Mean Squared Error: 0.21028899698021208, 	R2: 0.7925981274505838, 	Model score: 0.7925981274505838
n_estimators=3:
	Mean Squared Error: 0.1848344120051422, 	R2: 0.8177032383437348, 	Model score: 0.8177032383437348
n_estimators=4:
	Mean Squared Error: 0.1735920792742039, 	R2: 0.8287912215178597, 	Model score: 0.8287912215178597
n_estimators=5:
	Mean Squared Error: 0.1671274993191277, 	R2: 0.8351670472014754, 	Model score: 0.8351670472014754
n_estimators=6:
	Mean Squared Error: 0.16258598636752924, 	R2: 0.8396462082793019, 	Model score: 0.8396462082793019
n_estimators=7:
	Mean Squared Error: 0.15959214357214335, 	R2: 0.8425989476560561, 	Model score: 0.8425989476560561
n_estimators=8:
	Mean Squared Error: 0.1569536245635218, 	R2: 0.8452012415991712, 	Model score: 0.8452012415991712
n_estimators=9:
	Mean Squared Error: 0.15503735132541008, 	R2: 0.8470912057133568, 	Model score: 0.8470912057133568
n_estimators=10:
	Mean Squared Error: 0.15314115407118897, 	R2: 0.8489613694

Cross-Validation

In [17]:
#define number of splits
k_fold = KFold(n_splits=10)
for train_indices, test_indices in k_fold.split(X):
    print('Train: %s | test: %s' % (train_indices, test_indices))


Train: [ 2833  2834  2835 ... 28326 28327 28328] | test: [   0    1    2 ... 2830 2831 2832]
Train: [    0     1     2 ... 28326 28327 28328] | test: [2833 2834 2835 ... 5663 5664 5665]
Train: [    0     1     2 ... 28326 28327 28328] | test: [5666 5667 5668 ... 8496 8497 8498]
Train: [    0     1     2 ... 28326 28327 28328] | test: [ 8499  8500  8501 ... 11329 11330 11331]
Train: [    0     1     2 ... 28326 28327 28328] | test: [11332 11333 11334 ... 14162 14163 14164]
Train: [    0     1     2 ... 28326 28327 28328] | test: [14165 14166 14167 ... 16995 16996 16997]
Train: [    0     1     2 ... 28326 28327 28328] | test: [16998 16999 17000 ... 19828 19829 19830]
Train: [    0     1     2 ... 28326 28327 28328] | test: [19831 19832 19833 ... 22661 22662 22663]
Train: [    0     1     2 ... 28326 28327 28328] | test: [22664 22665 22666 ... 25494 25495 25496]
Train: [    0     1     2 ... 25494 25495 25496] | test: [25497 25498 25499 ... 28326 28327 28328]


In [18]:
[best_model.fit(X[train], y[train]).score(X[test], y[test]) for train, test in k_fold.split(X)]

[0.7940478550651568,
 0.8163159608675016,
 0.8629898155851882,
 0.8353905817384251,
 0.8676252691456386,
 0.8300412892409099,
 0.8139107352234508,
 0.8566500423836217,
 0.837955215439555,
 0.8257449292705563]

In [19]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    cv_score=cross_val_score(best_model, X, y.ravel(), cv=k_fold, n_jobs=-1)
    print(cv_score)

[0.79404786 0.81631596 0.86298982 0.83539058 0.86762527 0.83004129
 0.81391074 0.85665004 0.83795522 0.82574493]


In [20]:
#https://scikit-learn.org/stable/auto_examples/model_selection/plot_validation_curve.html
param_range = np.logspace(-6, -1, 5)
train_scores, test_scores = validation_curve(
    best_model,
    X,
    y,
    param_name="gamma",
    param_range=param_range,
    scoring="accuracy",
    n_jobs=2,
)
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.title("Validation Curve with SVM")
plt.xlabel(r"$\gamma$")
plt.ylabel("Score")
plt.ylim(0.0, 1.1)
lw = 2
plt.semilogx(
    param_range, train_scores_mean, label="Training score", color="darkorange", lw=lw
)
plt.fill_between(
    param_range,
    train_scores_mean - train_scores_std,
    train_scores_mean + train_scores_std,
    alpha=0.2,
    color="darkorange",
    lw=lw,
)
plt.semilogx(
    param_range, test_scores_mean, label="Cross-validation score", color="navy", lw=lw
)
plt.fill_between(
    param_range,
    test_scores_mean - test_scores_std,
    test_scores_mean + test_scores_std,
    alpha=0.2,
    color="navy",
    lw=lw,
)
plt.legend(loc="best")
plt.show()